# Advanced xlwings topics

# 1) Debugging

### RunPython

Let's create our hello world example again: `xlwings quickstart hello`

There's an easy way to debug your code that you run with RunPython, add something like this to the end of the file:

In [ ]:
if __name__ == '__main__':
    xw.Book('hello.xlsm').set_mock_caller()
    hello_xlwings()

## UDFs (Windows only)

To make use of xlwings' debug server, check `Debug UDFs` in the VBA settings and add the following lines to the end of your Python source file, then run the source file:

In [ ]:
if __name__ == '__main__':
    xw.serve()

 You can now set break points and print values. Depending on which debugger/IDE you use, you may need to run the program in "debug mode" (e.g. PyDev/PyCharm).

# 2) Real time feeds

### Write from Python to Excel

In [ ]:
import xlwings as xw
import datetime as dt
import time

sheet = xw.books.active.sheets.active
sheet['A1'].number_format = 'dd/mm/yy hh:mm:ss'  # include seconds

In [ ]:
# You need to interrupt the Kernel to stop
while True:
    sheet['A1'].value = dt.datetime.now()
    time.sleep(0.5)

<div class="alert alert-info">

**Note**: You need at least xlwings v0.11.7 for this to work correctly.

</div>

### Read Excel from Python

In [ ]:
import xlwings as xw
import time

sheet = xw.books.active.sheets.active

x = sheet['A3'].value

while True:
    y = sheet['A3'].value
    if x != y:
        x = y
        print(x)
    time.sleep(0.5)

# 3) Custom xlwings Extensions

We have seen in the beginner's course how to use the `sql` extension. As a refresher, xlwings extensions

* live in the add-in
* are immediatly usable without any further steps

### To create your own extension, you need to:

* Import a function
* Copy/Paste that code into the xlwings Extensions module of the add-in (password: xlwings)
* Change any occurrences of "ThisWorkook" to "ActiveWorkbook"

The only other thing that we need to take care of is the location of the Python source file. We have the following possibilities:

* Place it somewhere on your existing module search path (`sys.path`), e.g. in the site-packages folder (for more production-like setups, this can be done by creating a Python package and installing it via pip)
* Drop it in a "scripts" folder or similar and add that folder to the PYTHONPATH in the add-in or the config sheet

### => Let's turn the hello function from the quickstart project into an extension! 

# 4) Custom Converters 

## Quick Refresher

xlwings uses a default converter that handles 2d ranges as nested lists and takes care of the common formats like strings, datetime and numbers.

In [ ]:
import xlwings as xw
import pandas as pd
import numpy as np

wb = xw.Book()

In [ ]:
wb.sheets[0]["A1"].value = [[1, 'two'], ['three', None]]

In [ ]:
wb.sheets[0]["A1"].expand().value

## Writer your own converter!

Here are the steps to implement your own converter class:

* Inherit from `xlwings.conversion.Converter`
* Implement both a `read_value` and `write_value` method as static- or classmethod. Both functions have the same signature and return value: they expect and return the values in the format of the base converter (hence, if no `base` has been specified, `value` is a list of list as delivered by the default converter). On the other hand, the `options` dictionary will contain all keyword arguments specified in the `Range.options` method, e.g. when calling `Range('A1').options(myoption='some value')`. Here is the structure of `read_value` method:

```
@staticmethod
def read_value(value, options):
    myoption = options.get('myoption', default_value)
    # Implement your conversion here
    return value
```

* Optional: set a `base` converter to build on top of a builtin converter: `DictCoverter`, `NumpyArrayConverter`, `PandasDataFrameConverter`, `PandasSeriesConverter`
* Optional: register the converter: you can **(a)** register a type so that your converter becomes the default for this type during write operations and/or **(b)** you can register an alias that will allow you to explicitly call your converter by name instead of just by class name 

In [ ]:
from xlwings.conversion import Converter, PandasDataFrameConverter

class DataFrameDropna(Converter):

    base = PandasDataFrameConverter

    @staticmethod
    def read_value(df, options):
        dropna = options.get('dropna', False)
        if dropna:
            return df.dropna()
        else:
            return df

    @staticmethod
    def write_value(df, options):
        dropna = options.get('dropna', False)
        if dropna:
            df = df.dropna()
        return df

In [ ]:
sht2 = wb.sheets.add()

In [ ]:
df = pd.DataFrame([[1.,10.],[2.,np.nan], [3., 30.]])
df

## Built-in converter

In [ ]:
# Write
sht2.range('A1').value = df

In [ ]:
# Read
sht2.range('A1:C4').options(pd.DataFrame).value

## DataFrameDropna converter

In [ ]:
# Write
sht2.range('A7').options(DataFrameDropna, dropna=True).value = df

In [ ]:
# Read
sht2.range('A1:C4').options(DataFrameDropna, dropna=True).value

## Register an alias

In [ ]:
# Optional: Register an alias
DataFrameDropna.register('df')

In [ ]:
# Read using the alias
sht2.range('A1:C4').options('df', dropna=True).value

In [ ]:
# Write using the alias
sht2.range('A12').options('df', dropna=True).value = df

## Register DataFrameDropna as default converter

In [ ]:
# Optional: Override the builtin DataFrame converter
DataFrameDropna.register(pd.DataFrame)

In [ ]:
# Read
sht2.range('A1:C4').options(pd.DataFrame, dropna=True).value

In [ ]:
# Write
sht2.range('A17').options(dropna=True).value = df

# 5) REST API

* We recommend Insomnia (https://insomnia.rest/) or Postman (https://www.getpostman.com/) as REST API clients
* Make sure that you have Flask installed (`pip install Flask`)

In [ ]:
%matplotlib inline
import xlwings as xw
import requests
import pandas as pd

In [ ]:
base_url = 'http://127.0.0.1:5000'

In [ ]:
endpoint = '/books'
rv = requests.get(base_url + endpoint)

In [ ]:
rv.json()

In [ ]:
endpoint = '/book/timeseries.xlsx/sheets/sheet1/range/A1?expand=table'
rv = requests.get(base_url + endpoint)

In [ ]:
data = rv.json()
data

In [ ]:
df = pd.DataFrame(data['value'][1:],
                  columns=data['value'][0])
df = df.set_index('Date')
df.index = pd.to_datetime(df.index)
df

In [ ]:
df = df / df.iloc[0]

In [ ]:
df.plot()